In [1]:
debug = False

In [2]:
import numpy as np
import pandas as pd
from heapq import heappush, heappop
import time

In [3]:
initial_position = list(pd.read_csv('./inicial_medium.csv', delimiter=';', header=None).values[0])

# posicion(0,1)
initial_position[1] = initial_position[1].replace(' ', '')
initial_position

['OOOLLL', '', 'I']

In [4]:
pd.read_csv('./inicial_medium.csv', delimiter=';', header=None)

,0,1,2
0,OOOLLL,,I


In [5]:
operations = [
    'O',  #oveja
    'L',  #lobo
    'OO',  #ovejas
    'OL',  #lobo y oveja
    'LL'  #lobos
]

In [6]:
class Node:
    def __init__(self, value, parent=None):
        self.coste = None
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child, coste = None):
        node = Node(child, parent=self)
        if coste is not None:
            node.coste = coste
        self.children.append(node)
        return node

    def __eq__(self, other):
        return self.coste == other.coste

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self.coste < other.coste

    def __gt__(self, other):
        return self.coste > other.coste

    def __le__(self, other):
        return self.coste <= other.coste

    def __ge__(self, other):
        return self.coste >= other.coste


class Tree:
    def __init__(self, root):
        self.root = root


def find_path_to_root(objective_node):
    path = []
    current_node = objective_node
    while current_node is not None:
        path.insert(0, current_node.value)
        current_node = current_node.parent
    return path

In [7]:
def generate_children(estado, conjunto_de_operaciones):
    """
    Método que genera los posibles hijos de un estado dado
    Args:
        estado: Situación del problema.
        conjunto_de_operaciones: Operaciones que se pueden realizar

    Returns: Lista de posibles hijos (estados derivados del estado actual)

    """
    def change_position(operacion):
        """
        Método que cambia la posición de la barca
        Args:
            operacion: Animales que se quieren mover

        Returns: Nuevo estado del problema (animales a ambos lados y posición de la barca)


        """
        position = estado[2]
        if position == 'I':
            # Comprobar disponibilidad de animales. Si en el origen (actual) hay mas o mismos animales que los que se quieren mover se puede realizar la operación
            if all(estado[0].count(char) >= operacion.count(char) for char in set(operacion)):
                nueva_cadena_izquierda = estado[0]
                for char in operacion:
                    nueva_cadena_izquierda = nueva_cadena_izquierda.replace(char, '', 1)
                nuevo_destino_derecha = estado[1]
            else:
                if debug: print("No se puede realizar la operación {0} {1} {2}".format(estado, operacion, position))
                return None
        else:
            if all(estado[1].count(char) >= operacion.count(char) for char in set(operacion)):
                nueva_cadena_izquierda = estado[1]
                for char in operacion:
                    nueva_cadena_izquierda = nueva_cadena_izquierda.replace(char, '', 1)
                nuevo_destino_derecha = estado[0]
            else:
                if debug: print("No se puede realizar la operación {0} {1} {2}".format(estado, operacion, position))
                return None

        nuevo_destino_derecha = ''.join(sorted(nuevo_destino_derecha + operacion, reverse=True))

        # Comprobar estado válido
        if nueva_cadena_izquierda.count('L') > nueva_cadena_izquierda.count('O') > 0:
            return None
        if nuevo_destino_derecha.count('L') > nuevo_destino_derecha.count('O') > 0:
            return None

        # print("estado válido: ",nuevaCadena, ''.join(sorted(nuevoDestino + operation, reverse=True)), position)

        if position == 'I':
            return np.asarray([nueva_cadena_izquierda, nuevo_destino_derecha, 'D'])
        else:
            return np.asarray([nuevo_destino_derecha, nueva_cadena_izquierda, 'I'])

    children = []

    for op in conjunto_de_operaciones:
        new_position = change_position(op)
        if new_position is not None:
            children.append(new_position)

    return children

In [8]:
def movements_to_solution(solution_path):
    pasos = []

    for i in range(len(solution_path) - 1):
        estado_actual = solution_path[i]
        estado_siguiente = solution_path[i + 1]

        # Descubrir la dirección de la barca
        direccion = "-->" if estado_actual[2] == 'I' else "<--"

        # Descubrir qué animales se movieron
        if estado_actual[2] == 'I':
            # Animales desplazados de izquierda a derecha
            origen_antes = estado_actual[0]
            origen_despues = estado_siguiente[0]

            # Calcular diferencias
            ovejas_movidas = origen_antes.count('O') - origen_despues.count('O')
            lobos_movidos = origen_antes.count('L') - origen_despues.count('L')
        else:
            # Animales desplazados de derecha a izquierda
            origen_antes = estado_actual[1]
            origen_despues = estado_siguiente[1]

            ovejas_movidas = origen_antes.count('O') - origen_despues.count('O')
            lobos_movidos = origen_antes.count('L') - origen_despues.count('L')

        moved_animals = []
        if ovejas_movidas == 1:
            moved_animals.append("oveja")
        elif ovejas_movidas > 1:
            moved_animals.append(f"{ovejas_movidas} ovejas")

        if lobos_movidos == 1:
            moved_animals.append("lobo")
        elif lobos_movidos > 1:
            moved_animals.append(f"{lobos_movidos} lobos")

        pasos.append(
            "|".join(estado_actual[:2])
            + "|" + f"{', '.join(moved_animals)} {direccion}")

    return pasos

## A*

In [38]:
def heuristico(estado):
    #     """
    #     Método que calcula el valor heurístico de un estado
    #     Args:
    #         estado: Estado del problema
    #
    #     Returns: Valor heurístico del estado ( en este caso, número de animales en la orilla izquierda)
    #
    #     """
    num_ovejas = estado[0].count('O')
    num_lobos = estado[0].count('L')
    lado_barca = estado[2] == 'I'
    # return (num_ovejas + num_lobos) + ( 1 - lado_barca) * 1/4
    return (num_ovejas + num_lobos) / 2

In [27]:
def es_objetivo(estado):
    return estado[0] == ""

In [28]:
def construct_solution_a_estrella_river(tree):
    nodos_a_expandir = []
    heappush(nodos_a_expandir, (0, tree.root))
    nodos_visitados = set()


    while nodos_a_expandir:
        # current_node = heappop(nodos_a_expandir)
        _, current_node = heappop(nodos_a_expandir)
        current_position = current_node.value

        if tuple(current_position) in nodos_visitados:
            continue

        nodos_visitados.add(tuple(current_position))

        if es_objetivo(current_position):
            current_node = current_node.add_child(current_position)
            return tree, find_path_to_root(current_node), list(nodos_visitados)

        g = len(find_path_to_root(current_node)) - 1


        children = generate_children(current_position, operations)

        for c in children:
            if debug: print(c)
            if tuple(c) not in nodos_visitados:
                if debug: print(c)
                g_hijo = g + 1
                f = g_hijo + heuristico(c)

                if debug: print("para el nodo  {0} el valor de f es {1}".format(c, f))
                # hijo_completo = np.append(c, f)
                child = current_node.add_child(c,f)
                # nodos_a_expandir.put(child)
                # heappush(nodos_a_expandir, (child.coste,child))
                heappush(nodos_a_expandir, (f, child))

                # if es_objetivo(c):
                #     found = True
                #     path = find_path_to_root(child)
                #     break

    return tree, [], list(nodos_visitados)

In [29]:
start_time = time.time()

root = Tree(Node(initial_position))
arbol_obtenido, camino, nodos = construct_solution_a_estrella_river(root)

time.time() - start_time

0.001998424530029297

In [30]:

movements = movements_to_solution(camino)

print(" La solución se obtuvo en {0} + 1 pasos:".format(len(movements)-1))

movements

 La solución se obtuvo en 11 + 1 pasos:


['OOOLLL||oveja, lobo -->',
 'OOLL|OL|oveja <--',
 'OOOLL|L|2 lobos -->',
 'OOO|LLL|lobo <--',
 'OOOL|LL|2 ovejas -->',
 'OL|OOLL|oveja, lobo <--',
 'OOLL|OL|2 ovejas -->',
 'LL|OOOL|lobo <--',
 'LLL|OOO|2 lobos -->',
 'L|OOOLL|oveja <--',
 'OL|OOLL|oveja, lobo -->',
 '|OOOLLL| <--']

In [31]:
import time


def medir_tiempo(algoritmo, posicion_inicial_testbench, numero_de_ejecuciones=100000):
    _debug=False
    tiempos = []
    for _ in range(numero_de_ejecuciones) :
        inicio = time.perf_counter()  # Tiempo inicial
        _root = Tree(Node(posicion_inicial_testbench))
        _arbol_obtenido, _camino, _nodos = algoritmo(_root)
        fin = time.perf_counter()  # Tiempo final
        tempo = fin - inicio  # Tiempo total
        tiempos.append(tempo)

    media = np.mean(tiempos)
    minimo = np.min(tiempos)
    maximo = np.max(tiempos)

    unidad_de_escalado = 1000 # 1 segundo = 1000 milisegundos

    return media * unidad_de_escalado, minimo * unidad_de_escalado, maximo * unidad_de_escalado

In [32]:
datos_A_estrella = medir_tiempo(construct_solution_a_estrella_river, initial_position)
print("Algoritmo | Media {0: <10}| Mínimo {0: <9}| Máximo {0: <10} |".format(" "))
print("A*        | {0:.10f} ms | {1:.10f} ms | {2:.10f} ms  |".format(datos_A_estrella[0], datos_A_estrella[1], datos_A_estrella[2]))

Algoritmo | Media           | Mínimo          | Máximo            |
A*        | 0.4733392390 ms | 0.4056000034 ms | 65.5635000003 ms  |


In [39]:
def medir_nodos_generados(algoritmo, posicion_inicial_testbench, numero_de_ejecuciones=100000):
    """
    Mide el número de nodos generados de un algoritmo (BFS o A*).

    Args:
        algoritmo: Función del algoritmo a medir (construct_solution_BFS_river o construct_solution_a_estrella_river).
        posicion_inicial_testbench: Estado inicial do problema.

    Returns:
        nodos: Número de nodos generados.
    """
    nodos = []
    for _ in range(numero_de_ejecuciones) :
        _root = Tree(Node(posicion_inicial_testbench))
        _arbol_obtenido, _camino, _nodos = algoritmo(_root)
        nodos.append(len(_nodos))

    return np.mean(nodos)

In [41]:
medir_nodos_generados(construct_solution_a_estrella_river, initial_position)

15.0